# Mandatory Assignment
**Torger Bocianowski**

## Task 1

### Sets

$S=$ suppliers

$P=$ plants

### Parameters

- Distance from supplier to plant (km)
- Cost per unit on a road from supplier to plant
- Truck cost 
- Truck capacity
- Plant cost
- Plant capacity
- Yield per unit at a plant
- Supply of biomass in tons for a supplier

### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sp}=$ tons of biomass transported from supplier $s$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks used.

### Formulating the Objective Function

$C_p=$ cost of opening a plant.

$d_{sp}=$ distance driven (km) from supplier $s$ to plant $p$.

$y_p=$ yield per unit at a plant $p$.

$L, U=$ loading / unloading cost

<br></br>

**Cost components**

$\sum_p C_p\cdot b_p$ whether or not we decide to build a plant $p$ and multiplying it by the opening cost of $p$.

$\sum_{sp}u_{sp}x_{sp}d_{sp}$ the unit cost times the amount biomass transported times the distance driven.

$(L+U)t$, multiplying the number of trucks used by the constant load / unload cost.

<br></br>

**Objective function**

Minimize Cost: $\space\sum_p C_p\cdot b_p+\sum_{sp}u_{sp}x_{sp}d_{sp}+(L+U)t$

### Constraints

- $\sum_{sp}x_{sp}y_p\geq500\space000\space000$

- $\sum_s x_{sp}y_p\leq cap_p\quad\forall p\in P$

- $\frac{x_{sp}}{cap_t}\leq t\quad\forall s\in S\quad\forall p\in P$

- $x_{sp} >= 0\quad\forall s\in S\quad\forall p\in P$

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Dataframes
suppliers_df = pd.read_csv("data/suppliers.csv")
plants_df = pd.read_csv("data/plants.csv")
roads_df = pd.read_csv("data/roads_s_p.csv")

# Sets
suppliers = suppliers_df.set_index("supplier")
suppliers = suppliers.to_dict(orient="index")

plants = plants_df.set_index("plant")
plants = plants.to_dict(orient="index")

### Defining the Model

In [3]:
m = gp.Model("SupplyChain")
gp.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-24


### Parameters

In [4]:
def param_from(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "plant", param]]
    df = df.set_index(["supplier", "plant"])
    df = df.to_dict(orient="index")
    return {(s, p): df[s, p][param] for s in suppliers for p in plants}

dist_s_p = param_from(roads_df, "dist_s_p")
unit_cost_s_p = param_from(roads_df, "cost_per_unit_s_p")
truck_cost_s_p = param_from(roads_df, "truck_cost_s_p")
truck_cap_s_p = param_from(roads_df, "truck_cap_s_p")

### Decision Variables

In [5]:
build = m.addVars(plants, vtype=GRB.BINARY, name="build")
trucks = m.addVar(vtype=GRB.INTEGER, name="trucks")
biomass = m.addVars(suppliers, plants, name="biomass")

### Objective Function

In [6]:
load, unload = 5000, 5000

m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build[p]
        for p in plants
    )
    + gp.quicksum(
        unit_cost_s_p[s, p] * biomass[s, p] * dist_s_p[s, p]
        for s in suppliers
        for p in plants
    )
    + (load + unload) * trucks, 
    GRB.MINIMIZE,
)

### Constraints

In [7]:
PRODUCTION = 500_000_000

m.addConstrs(
    (
        gp.quicksum(biomass[s, p] for p in plants) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstrs(
    (
        gp.quicksum(biomass[s, p] for s in suppliers) <= plants[p]["plant_cap"] * build[p]
        for p in plants
    ),
    name="capacity",
)

m.addConstr(
    (
        gp.quicksum(biomass[s, p] * plants[p]["yield_per_unit"] 
                    for s in suppliers 
                    for p in plants
                    ) >= PRODUCTION
    ),
    name="production",
)

m.addConstrs(
    (
        biomass[s, p] / truck_cap_s_p[s, p] <= trucks
        for s in suppliers
        for p in plants
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass[s, p] >= 0
        for s in suppliers
        for p in plants
    ),
    name="non_negative_biomass",
)

m.setParam("OutputFlag", 0)

### Solve the Model

In [10]:
m.setParam("OutputFlag", 1)
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85258 rows, 42586 columns and 254675 nonzeros
Model fingerprint: 0x07a443fd
Variable types: 42418 continuous, 168 integer (167 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+08]
  Objective range  [1e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 41832 rows, 41584 columns, 207247 nonzeros

Continuing optimization...


Cutting planes:
  MIR: 1

Explored 1 nodes (2098 simplex iterations) in 0.03 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 8.45859e+09 8.46012e+09 8.48423e+09 ... 9.0684e+09

Optimal solution found (tolerance 1.00e-04)
Best objective 8.458593644092e+09, best bound 8.458593644092e+09, 

In [11]:
locations = [p for p in plants if build[p].x == 1]
total_biomass = sum(biomass[s, p].x for s in suppliers for p in plants)
formatted_total_biomass = f"{total_biomass:,.0f}"
formatted_total_cost = f"${m.objVal:,.2f}"

print("RESULTS")
print(f"({len(locations)}) Locations: {locations}")
print(f"Total biomass: {formatted_total_biomass} Mg")
print(f"Trucks needed: {trucks.x}")
print(f"Total cost: {formatted_total_cost}")


RESULTS
(11) Locations: [541, 9063, 9071, 9102, 9107, 9155, 9178, 9183, 9203, 10058, 10066]
Total biomass: 2,155,172 Mg
Trucks needed: 91.0
Total cost: $8,458,593,644.09


### Interpreting the Results

As shown above, 
- there are **11 locations** at which we want to **build plants**, 
- we need **91 trucks** and 
- the **total cost** is almost **8.5 billion** dollars

## Task 2

### Sets
$S=$ suppliers

$P=$ plants

$H=$ hubs

### Parameters

- Hub cost
- Hub capacity
- Truck cost from supplier to hub
- Truck capacity from supplier to hub
- Train cost from hub to plant
- Train capacity from hub to plant
- Distance from supplier to hub
- Distance from hub to supplier
- Cost per unit on a road from supplier to hub
- Cost per unit on a road from hub to plant


### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$b_h\in\{0,1\}=$ binary variable, whether a hub at location $h$ should be built or not.

$x_{sh}=$ tons of biomass transported from supplier $s$ to hub $h$.

$x_{hp}=$ tons of biomass transported from hub $h$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks to be used.

$r\in\mathbb{Z}=$ integer variable, the number of trains to be used.

### Formulating the Objective Function

$O_p=$ constant opening cost of a plant $p$.

$O_h=$ constant opening cost of a hub $h$.

$d_{sh}=$ distance driven (km) from supplier $s$ to hub $h$ by road.

$d_{hp}=$ distance driven (km) from hub $h$ to plant $p$ by train.

$L_t, U_t=$ truck loading / unloading cost

$L_r, U_r=$ train loading / unloading cost

<br></br>

**Cost components**

$\sum_p O\cdot b_p$ whether or not we decide to build a plant and multiplying it by the opening cost.

$\sum_{sp}c_{sp}x_{sp}d_{sp}$ the transportation cost times the amount transported times the distance driven.

$(L+U)t$, multiplying the number of trucks used by the constant load / unload cost.

<br></br>

**Objective function**

Minimize Cost: $\space
\sum_p O_p\cdot b_p+\sum_h O_h\cdot b_h+
\sum_{sh}c_{sh}x_{sh}d_{sh}+\sum_{hp}c_{hp}x_{hp}d_{hp}
(L_t+U_t)t+(L_h+U_h)h$

### Constraints

<p style="color: red;">Flow balance?</p>

In [12]:
# Dataframes
hubs_df = pd.read_csv("data/hubs.csv")
railroads_df = pd.read_csv("data/railroads_h_p.csv")
roads_df = pd.read_csv("data/roads_s_h.csv")

# Sets
hubs = hubs_df.set_index("hub")
hubs = hubs.to_dict(orient="index")

### Defining the Model

In [13]:
m = gp.Model("SupplyChain2")
gp.setParam("OutputFlag", 0)

### Parameters

In [14]:
# Helper functions
def sh_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "hub", param]]
    df = df.set_index(["supplier", "hub"])
    df = df.to_dict(orient="index")
    return {(s, h): df[s, h][param] for s in suppliers for h in hubs}

def hp_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["hub", "plant", param]]
    df = df.set_index(["hub", "plant"])
    df = df.to_dict(orient="index")
    return {(h, p): df[h, p][param] for h in hubs for p in plants}

# params from roads_s_h.csv
dist_s_h = sh_param(roads_df, "dist_s_h")
unit_cost_s_h = sh_param(roads_df, "cost_per_unit_s_h")
truck_cap_s_h = sh_param(roads_df, "truck_cap_s_h")

# params from railroads_h_p.csv
dist_h_p = hp_param(railroads_df, "dist_h_p")
unit_cost_h_p = hp_param(railroads_df, "cost_per_unit_h_p")
train_cap_h_p = hp_param(railroads_df, "train_cap_h_p")

### Decision Variables

In [15]:
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

### Objective Function

In [16]:
truck_L, truck_U = 5000, 5000
train_L, train_U = 30_000, 30_000

m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + ((truck_L + truck_U) * trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + ((train_L + train_U) * trains[h, p])
        for h in hubs
        for p in plants
    ),
    GRB.MINIMIZE,
)

### Constraints

In [20]:
PRODUCTION = 500_000_000

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] * build_hub[h] for h in hubs) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    (
        gp.quicksum(biomass_h_p[h, p] * plants[p]["yield_per_unit"]
                    for h in hubs 
                    for p in plants
                    ) >= PRODUCTION
    ),
    name="production",
)

m.addConstrs(
    (
        biomass_s_h[s, h] <= trucks[s, h] * truck_cap_s_h[s, h]
        for s in suppliers
        for h in hubs
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass_h_p[h, p] <= trains[h, p] * train_cap_h_p[h, p]
        for h in hubs
        for p in plants
    ),
    name="trains",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers) <= hubs[h]["hub_cap"] * build_hub[h]
        for h in hubs
    ),
    name="hub_capacity",
)


m.addConstrs(
    (
        gp.quicksum(biomass_h_p[h, p] for h in hubs) <= plants[p]["plant_cap"] * build_plant[p]
        for p in plants
    ),
    name="plant_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers)
        == gp.quicksum(biomass_h_p[h, p] for p in plants)
        for h in hubs
    ),
    name="balance_flow",
)

gp.setParam("OutputFlag", 0)

### Solve the Model

In [21]:
m.optimize()

### Solve the Model

In [22]:
plants_locations = [p for p in plants if build_plant[p].x == 1]
hubs_locations = [h for h in hubs if build_hub[h].x == 1]

total_biomass_s_h = sum(biomass_s_h[s, h].x for s in suppliers for h in hubs)
total_biomass_s_h = f"{total_biomass_s_h:,.0f}"

total_biomass_h_p = sum(biomass_h_p[h, p].x for h in hubs for p in plants)
total_biomass_h_p = f"{total_biomass_h_p:,.0f}"

if total_biomass_s_h != total_biomass_h_p:
    print(f"Biomass from s to h: {total_biomass_s_h} Mg")
    print(f"Biomass from h to p: {total_biomass_h_p} Mg")
    raise Exception("Biomass from s to h must be equal to biomass from h to p")


print("RESULTS\n")
print(f" ({len(plants_locations)}) Plants: {plants_locations}")
print(f"({len(hubs_locations)})   Hubs: {hubs_locations[:10]}")
print(f"\t     {hubs_locations[10:20]}")
print(f"\t     {hubs_locations[20:]}\n")
print(f"Total biomass: {total_biomass_s_h} Mg")

print(f"Trucks needed: {len(trucks.keys())}")
print(f"Trains needed: {len(trains.keys())}")
print(f"\nTotal cost: ${m.objVal:,.2f}")

RESULTS

 (8) Plants: [541, 9047, 9060, 9091, 9178, 9183, 9203, 10066]
(31)   Hubs: [17201, 17218, 17359, 17372, 17395, 17404, 17447, 17466, 17507, 17592]
	     [17620, 17679, 17717, 17784, 17792, 17822, 17829, 17896, 17931, 17934]
	     [17942, 17943, 18029, 18042, 18063, 18082, 18127, 18286, 18288, 18294, 18303]

Total biomass: 2,155,172 Mg
Trucks needed: 8382
Trains needed: 5511

Total cost: $5,135,420,807.26


### Interpreting the Results

As shown above, 
- there are **8 locations** at which we want to **build plants**, 
- there are **31 locations** at which we want to **build hubs**, 
- we need **8382 trucks** and **5511** trains,
- the **total cost** is about **5.1 billion** dollars

## Task 3

### Sets

$S=$ suppliers

$P=$ plants

### Parameters

- Distance from supplier to plant (km)
- Cost per unit on a road from supplier to plant
- Truck cost 
- Truck capacity
- Plant cost
- Plant capacity
- Yield per unit at a plant
- Supply of biomass in tons for a supplier

### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sp}=$ tons of biomass transported from supplier $s$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks used.

### Defining the Model

In [24]:
m = gp.Model("SupplyChain3")
gp.setParam("OutputFlag", 0)

### Parameters

In [26]:
# Same as before, but with a third party with unlimited supply
unit_cost_third_party = 2000

### Decision Variables

In [27]:
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")
third_party_suppliers = m.addVar(vtype=GRB.INTEGER, name="third_party_suppliers")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

### Objective Function

In [28]:
truck_L, truck_U = 5000, 5000
train_L, train_U = 30_000, 30_000
third_party_unit_cost = 2000

# same as before but with third party with unlimited supply to hubs and a unit cost of 2000
# the location of the third parties does not matter
m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + ((truck_L + truck_U) * trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + ((train_L + train_U) * trains[h, p])
        for h in hubs
        for p in plants
    )
    + third_party_unit_cost * third_party_suppliers,
    GRB.MINIMIZE,
)

### Constraints

In [ ]:
# same as before but 
PRODUCTION = 800_000_000